# Corresponding Notebook for Homework

### Initiations of Constants, Libraries, and Environments

In [30]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib
#%matplotlib notebook

In [31]:
R = 1
ds = 0.05
a = 0.7*R
mini_r = R/5
V = 10
x_e = 80.1
e_0 = 8.85e-12

In [32]:
x = np.arange(-2*R,2*R+ds,ds)
y = np.arange(-2*R,2*R+ds,ds)
z = np.arange(-2*R,2*R+ds,ds)
X,Y,Z = np.meshgrid(x,y,z,indexing='ij')  

## Method of Relaxation

This is very similar to the Prep Activity, but instead there are two objects that need to re-initialized for their potentials.

In [33]:
def relax(volt,sphere_1,sphere_2, num):
    count = 0
    while(num != count):

        #grabs the old voltage 
        old_volt = volt 
        
        #re-inforces the voltage boundary condition, or else it washes out                                                                                                      
        volt[sphere_1] = V 
        volt[sphere_2] = -V                                                                                                         
        for i in range(1,volt.shape[0]-1):
            for j in range(1,volt.shape[1]-1):
                for k in range(1,volt.shape[2]-1):
                    volt[i,j,k] =  (old_volt[i+1,j,k] +  old_volt[i-1,j,k] +  old_volt[i,j+1,k] +  
                                    old_volt[i,j-1,k] + old_volt[i,j,k+1] +  old_volt[i,j,k-1])/6
        count = count + 1
    return volt

Here, there are two spherical objects. Each are fixed on the xy-plane, and are separated by a distance of 2a

In [34]:
sphere_1_index = np.where(((X-a)**2 + (Y)**2 + Z**2 < mini_r**2))
sphere_2_index = np.where(((X+a)**2 + (Y)**2 + Z**2 < mini_r**2))

## Potential Calculations

This creates a Mask for the potential for both objects

In [35]:
volt_hw = np.zeros((x.size,y.size,z.size),dtype = np.float64)
volt_mask = np.zeros((x.size,y.size,z.size),dtype = bool)
volt_mask[sphere_1_index] = True
volt_mask[sphere_2_index] = True

perm_index = np.where(X**2 + Y**2 + Z**2 <= R**2) 
other_index = np.where(X**2 + Y**2 + Z**2 > R**2)    

voltage = relax(volt_hw,sphere_1_index,sphere_2_index, 100)

**Contour Plot at Constant Z**

In [38]:
_len = voltage.shape[2]
mid_len = int(round(_len ,0)/2)
plt.title(f'Equipotential Contour Plot of Dipole Moment at Constant Z')
plt.contourf(X[:,:,mid_len],Y[:,:,mid_len],voltage[:,:,mid_len],levels = 5000,cmap = 'copper')
plt.xlabel('X')
plt.ylabel('Y')
plt.colorbar(label = 'potential [V]',ticks = np.linspace(-V,V,V*2))
plt.tight_layout()
plt.show()

Locator attempting to generate 4995 ticks ([-9.988, ..., 9.988]), which exceeds Locator.MAXTICKS (1000).


**Contour Plot at Constant X**

In [11]:
plt.title(f'Equipotential Contour Plot of Dipole Moment at Constant X')
plt.contourf(X[:,:,mid_len],Y[:,:,mid_len],voltage[mid_len,:,:],levels = 5000,cmap = 'copper')
plt.xlabel('Y')
plt.ylabel('Z')
plt.colorbar(label = 'potential [V]',ticks = np.linspace(-V,V,V*2))
plt.tight_layout()
plt.show()

Locator attempting to generate 4722 ticks ([-0.023604999999999998, ..., 0.0]), which exceeds Locator.MAXTICKS (1000).


**Contour Plot at Constant Y**

In [12]:
plt.title(f'Equipotential Contour Plot of Dipole Moment at Constant Y')
plt.contourf(X[:,:,mid_len],Y[:,:,mid_len],voltage[:,mid_len,:],levels = 5000,cmap = 'copper')
plt.xlabel('X')
plt.ylabel('Z')
plt.colorbar(label = 'potential [V]',ticks = np.linspace(-V,V,V*2))
plt.tight_layout()
plt.show()

Locator attempting to generate 4995 ticks ([-9.988, ..., 9.988]), which exceeds Locator.MAXTICKS (1000).


## Creation of the Electric Field

In [13]:
V_in = np.zeros((voltage.shape),dtype = np.float64) 
V_in[perm_index] = voltage[perm_index] 

**Electric Field Inside the Dielectric**

In [14]:
E_in = np.gradient((V_in) * -np.ones((V_in.shape),dtype = np.float64) / (np.ones((V_in.shape),dtype = np.float64)*x_e))                      
x_E_in = E_in[0]                                                                                                                       
y_E_in = E_in[1]                                                                                                                             
z_E_in = E_in[2]     

**Electric Field Outside the Dielectric**

In [15]:
E_cube_out =  np.gradient(voltage) * -np.ones((voltage.shape),dtype = np.float64)        
x_E_out = E_cube_out[0]                                                                
y_E_out = E_cube_out[1]                                                                  
z_E_out = E_cube_out[2]   

**3D Electric Field Plot**

In [16]:
ax = plt.figure().add_subplot(projection='3d')
ax.quiver(X[perm_index], Y[perm_index], Z[perm_index], x_E_in[perm_index], y_E_in[perm_index], z_E_in[perm_index], length=0.1,arrow_length_ratio=0.5,color = 'black')
ax.quiver(X[other_index], Y[other_index], Z[other_index], x_E_out[other_index], y_E_out[other_index], z_E_out[other_index], length=.1,arrow_length_ratio=0.5,color = 'black')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_aspect('equal')
ax.set_xlim(-2*R,2*R)
ax.set_ylim(-2*R,2*R)
plt.show()

**2D Electric Field Plot at Constant X**

In [17]:
#slices the electric field within dielectric at midpoint
y_in_pl_const_x = y_E_in[mid_len,:,:]        
z_in_pl_const_x = z_E_in[mid_len,:,:]  

#slices the electric field outside dielectric at midpoint 
y_out_pl_const_x = y_E_out[mid_len,:,:]     
z_out_pl_const_x = z_E_out[mid_len,:,:]

#slices the positions
X_const_x = X[mid_len,:,:]                        
Y_const_x = Y[mid_len,:,:]
Z_const_x = Z[mid_len,:,:]

#grabs 2D indices of the slices that is within the dielectric
new_index_const_x = np.where(X_const_x**2 + Y_const_x**2 + Z_const_x**2 < R)    #grabs 2D indices of the slices that is within the dielectric

#grabs 2D indices of the slices that is outside the dielectric
out_index_const_x = np.where(X_const_x**2 + Y_const_x**2 + Z_const_x**2 > R)    #grabs 2D indices of the slices that is outside the dielectric

#relating color of plot to magnitude of vectors
color_in_const_x = np.sqrt(y_in_pl_const_x**2 + z_in_pl_const_x**2)      #related color to magnitude
color_out_const_x = np.sqrt(y_out_pl_const_x**2 + z_out_pl_const_x**2)   #related color to magnitude

fig, ax = plt.subplots()
#ax.set_title(f"Cross-sectional Cut of Electric Field Cone at Constant X with potential {V} [V] and height {cone_height} [m] and upper radius {round(cone_radius,3)} [m]")
ax.set_aspect('equal')
ax.set_facecolor('black')
ax.set_xlabel('X')
ax.set_ylabel('Y')
quiver_1 = ax.quiver(Y_const_x[new_index_const_x],Z_const_x[new_index_const_x],y_in_pl_const_x[new_index_const_x],z_in_pl_const_x[new_index_const_x],color_in_const_x[new_index_const_x],units='xy',angles='xy',scale_units='xy',headlength=5,cmap='hot')    
quiver_2 = ax.quiver(Y_const_x[out_index_const_x],Z_const_x[out_index_const_x],y_out_pl_const_x[out_index_const_x],z_out_pl_const_x[out_index_const_x],color_out_const_x[out_index_const_x],units='xy',angles='xy',scale_units='xy',headlength=5,cmap='hot')
cbar = plt.colorbar(quiver_1,ticks = np.linspace(0,np.max(color_in_const_x),5))      #creates colorbar. Used quiver 1 because max values are with it
cbar.set_label(r'$|\mathrm{\vec{E}}| \quad [\mathrm{N C^{-1}}]$')
ax.add_patch(plt.Circle((0, 0), R, color='black', fill=False,linewidth = 5))
ax.add_patch(plt.Circle((-a, 0), mini_r, color='green', fill=False,linewidth = 2))
ax.add_patch(plt.Circle((a, 0), mini_r, color='green', fill=False,linewidth = 2))
ax.set_xlim(-2*R,2*R)
ax.set_ylim(-2*R,2*R)
plt.show()

(-2.0, 2.0)

**2D Electric Field Plot at Constant Z**

In [18]:
#slices the electric field within dielectric at midpoint
x_in_pl_const_z = x_E_in[:, :, mid_len]        
y_in_pl_const_z = y_E_in[:, :, mid_len]  

#slices the electric field outside dielectric at midpoint 
x_out_pl_const_z = x_E_out[:, :, mid_len]     
y_out_pl_const_z = y_E_out[:, :, mid_len]

#slices the positions
X_const_z = X[:, :, mid_len]                         
Y_const_z = Y[:, :, mid_len]
Z_const_z = Z[:, :, mid_len]

#grabs 2D indices of the slices that is within the dielectric
new_index_const_z = np.where(X_const_z**2 + Y_const_z**2 + Z_const_z**2 < R)    #grabs 2D indices of the slices that is within the dielectric

#grabs 2D indices of the slices that is outside the dielectric
out_index_const_z = np.where(X_const_z**2 + Y_const_z**2 + Z_const_z**2 > R)    #grabs 2D indices of the slices that is outside the dielectric

#related color to magnitude
color_in_const_z = np.sqrt(x_in_pl_const_z**2 + y_in_pl_const_z**2)      
color_out_const_z = np.sqrt(x_out_pl_const_z**2 + y_out_pl_const_z**2)   

fig, ax = plt.subplots()
ax.set_title(f"Cross-sectional Cut of Electric Field Cone at Constant Z with potential {V} [V] and objects' radius {round(a,4)} [m]")
ax.set_aspect('equal')
ax.set_facecolor('black')
ax.set_xlabel('X')
ax.set_ylabel('Y')
quiver_1 = ax.quiver(X_const_z[new_index_const_z],Y_const_z[new_index_const_z],x_in_pl_const_z[new_index_const_z],y_in_pl_const_z[new_index_const_z],color_in_const_z[new_index_const_z],units='xy',angles='xy',scale_units='xy',headlength=5,cmap='hot')   
quiver_2 = ax.quiver(X_const_z[out_index_const_z],Y_const_z[out_index_const_z],x_out_pl_const_z[out_index_const_z],y_out_pl_const_z[out_index_const_z],color_out_const_z[out_index_const_z],units='xy',angles='xy',scale_units='xy',headlength=5,cmap='hot')
cbar = plt.colorbar(quiver_1,ticks = np.linspace(0,np.max(color_in_const_z),5))      #creates colorbar. Used quiver 1 because max values are with it
cbar.set_label(r'$|\mathrm{\vec{E}}| \quad [\mathrm{N C^{-1}}]$')
ax.add_patch(plt.Circle((-a, 0), mini_r, color='white', fill=False))
ax.add_patch(plt.Circle((a, 0), mini_r, color='white', fill=False))
ax.add_patch(plt.Circle((0, 0), R, color='white', fill=False,linewidth = 1))
ax.set_xlim(-2*R,2*R)
ax.set_ylim(-2*R,2*R)
plt.show()

(-2.0, 2.0)

### Charge Density Calculations

In [19]:
pos_mask = np.zeros((x.size,y.size,z.size),dtype = bool)     
pos_mask[sphere_1_index] = True 
pos_mask[sphere_2_index] = True                                  
pos_mask[1:-1,1:-1,1:-1] = (pos_mask[2:,1:-1,1:-1] | pos_mask[0:-2,1:-1,1:-1] 
| pos_mask[1:-1,2:,1:-1] | pos_mask[1:-1,0:-2,1:-1] 
| pos_mask[1:-1,1:-1,2:] | pos_mask[1:-1,1:-1,0:-2])
pos_mask[sphere_1_index] = False  
pos_mask[sphere_2_index] = False  

In [20]:
x_E_bound_in = x_E_in[pos_mask]
y_E_bound_in = y_E_in[pos_mask]
z_E_bound_in = z_E_in[pos_mask]

In [21]:
mag_E_bound_in_one = np.sqrt(x_E_bound_in**2 +
                                    y_E_bound_in**2 + z_E_bound_in**2)
sig_b = -mag_E_bound_in_one*e_0*x_e

**Inner Bound Surface Charge Density**

In [22]:
ax = plt.figure().add_subplot(projection='3d')
plt.title(f'Surface Bound Charge Density at Inner Boundary with Potential {round(V,4)} [V]')
plot = ax.scatter(X[pos_mask],Y[pos_mask],Z[pos_mask],marker = 'o',cmap = 'hot' ,c= sig_b, s = 1000,alpha = 1)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_aspect('equal')
cbar = plt.colorbar(plot) 
cbar.set_label(r'$\mathrm{\sigma \quad [\frac{C}{m^2}]}$')    
plt.show()

**Masking for just the inside edge of the dielectric**

In [23]:
scale = 2  
sphere_perm = np.where(((np.abs(X))**2 + (np.abs(Y))**2 + (np.abs(Z))**2 < (R)**2) & ((np.abs(X)+scale*ds)**2 + (np.abs(Y)+scale*ds)**2 + (np.abs(Z)+scale*ds)**2 > (R+scale*ds)**2))

**Electric Field Just Outside the dielectric**

In [24]:
x_E_bound_out = x_E_out[sphere_perm]
y_E_bound_out = y_E_out[sphere_perm]
z_E_bound_out = z_E_out[sphere_perm]

In [25]:
dot_x = x_E_bound_out * X[sphere_perm]
dot_y = y_E_bound_out * Y[sphere_perm]
dot_z = z_E_bound_out * Z[sphere_perm]
sig_out_b = (e_0*x_e/R) * (dot_x + dot_y + dot_z)

**Outer Surface Bound Charge Denisty**

In [28]:
ax = plt.figure().add_subplot(projection='3d')
plt.title(f'Surface Bound Charge Density at Outer Boundary with Potential {round(V,4)} [V]')
plot = ax.scatter(X[sphere_perm],Y[sphere_perm],Z[sphere_perm],marker = 'o',cmap = 'hot' ,c= sig_out_b, s = 1000,alpha = 1)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_aspect('equal')
cbar = plt.colorbar(plot) 
cbar.set_label(r'$\mathrm{\sigma \quad [\frac{C}{m^2}]}$') 
plt.show()

**Surface Free Charge Density**

In [29]:
sig_f = mag_E_bound_in_one*e_0
ax = plt.figure().add_subplot(projection='3d')
plt.title(f'Surface Free Charge Density at surface with Potential {round(V,4)} [V]')
plot = ax.scatter(X[pos_mask],Y[pos_mask],Z[pos_mask],marker = 'o',cmap = 'hot' ,c= sig_f, s = 1000,alpha = 1)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_aspect('equal')
cbar = plt.colorbar(plot)
cbar.set_label(r'$\mathrm{\sigma \quad [\frac{C}{m^2}]}$') 
plt.show()